# Evaluation prearation

## Environment setup

In [1]:
!pip install transformers
!pip install boto3
!git clone https://github.com/NVIDIA/apex
!pip install jiwer==2.2.0
!pip install -U nltk
!pip install rouge/requirements.txt
!pip install rouge-score
%cd apex
!pip install -v --no-cache-dir ./
%cd .. 


fatal: destination path 'apex' already exists and is not an empty directory.
Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.6.2)
ERROR: Invalid requirement: 'rouge/requirements.txt'
Hint: It looks like a path. File 'rouge/requirements.txt' does not exist.
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-52qkbt5a
Created temporary directory: /tmp/pip-req-tracker-0pojv8_m
Created requirements tracker '/tmp/pip-req-tracker-0pojv8_m'
Created temporary directory: /tmp/pip-install-ou0y4ipu
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-51vtyh8q
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-0pojv8_m'
    Running setup.py (path:/tmp/pip-req-build-51vtyh8q/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-51vtyh8q/pip-egg-info/apex.egg-info
    wr

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
# data preprocessing
import pandas as pd
import re
import numpy as np
import json
import h5py

# control
import argparse
import logging
from tqdm import trange

# pytorch
import torch
print(torch.__version__)
import torch.nn.functional as F

# transformers
from transformers import GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BartForCausalLM, BartTokenizer,BartConfig

1.8.1+cu101


In [4]:
# fine-tuning model with bart
# logging info
import logging
logger = logging.getLogger(__name__)

import glob
import logging
import random
import gc
import boto3
import shutil

In [5]:
from tqdm import tqdm, trange
import math
import os
from dataclasses import dataclass, field
from typing import Optional

from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BartForCausalLM, BartTokenizer,BartConfig

## data loading

In [6]:
# load an independent test set that previously saved
df_eval = pd.read_csv('./gdrive/MyDrive/COMP0087_v7/350k_GPT-2_Score.csv')
df_eval_ner = df_eval["NER"]  #NER
df_eval_dir = df_eval["directions"]

In [7]:
df_eval_dir

0     ["Blend 1/2 quart dry milk, 14 ounces club sod...
1     ["Cook orzo in lightly salted water as package...
2     ["Mix corn, salt and milk together.", "Beat eg...
3     ["Mix cornstarch, sugar and salt.", "Add hot w...
4     ["Mix taco seasoning and refried beans to desi...
                            ...                        
95    ["Beat egg substitute and add, in order, spice...
96    ["Pour all ingredients over chicken breasts in...
97    ["Put spices and sugar in basket of 8-cup perc...
98    ["Melt chocolate in boiling water.", "Cool.", ...
99    ["Bring milk, shortening, cocoa, and sugar to ...
Name: directions, Length: 100, dtype: object

## generation functions

In [8]:
def init_tokenizer(tokenizer_type = "facebook/bart-base"):
  
  tokenizer = BartTokenizer.from_pretrained(tokenizer_type)
  special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
      ]
  }
  tokenizer.add_special_tokens(special_tokens)
  return tokenizer

In [9]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, context, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        for _ in range(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated

In [10]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):

    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    context_tokens = tokenizer.encode(prepared_input)[0:-1]
    out = sample_sequence(
                model=model,
                context=context_tokens,
                tokenizer=tokenizer,
                length=800,
                device = 'cuda'
            )
    out = out[0, len(context_tokens):].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      # print(text)
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      # print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      # print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

## cosine similarity

In [11]:
#func of cos_sim
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [12]:
#cosine_similarity

def COS_SIM(df_eval_dir,directions):
  """
  df_eval_dir := gold_standard recipes
  directions  := corresponding recipes """

  avg = 0

  for i in range(len(directions)):
    best = 0
    for j in range(len(directions[i])):
      cos = get_result(" ".join(eval(df_eval_dir[i])),
                      " ".join(directions[i][j]))
      best = max(best, cos)

    avg += best

  avg = avg/len(directions)
  return avg

## Bleu Score

In [13]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor



In [14]:
# Helper Func.
def list_to_words(recipe):
  words = []
  for i in recipe:
    words += i.split()

  return words
# bleu score
def bleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    smoothie = SmoothingFunction().method5
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie, weights=(1, 0, 0, 0))
    return score_ref_a


In [15]:
def bleu_cal(df_eval_dir, directions):
  bleu_avg = []
  for i in range(len(directions)):
    bleu_avg.append(bleu_score(df_eval_dir[i], directions[i]))
  return np.mean(bleu_avg)


## ROUGE

In [16]:
from rouge_score import rouge_scorer

def list_to_string(start_list):
  string=''.join([str(item) for item in start_list])
  return string

def rouge_cal(df_eval_dir, directions, rouge_type = 'rougeL'):
  rough_avg = []
  scorer = rouge_scorer.RougeScorer([rouge_type], use_stemmer=True)

  for i in range(len(df_eval_dir)):
    for j in range(10):
      scores= scorer.score(df_eval_dir[i], list_to_string(directions[i][j]))
      rough_avg.append(list(scores.values())[0][0])
  return np.mean(rough_avg)



# model evaluation

## BART1+GRU

In [18]:
# model = BartForCausalLM.from_pretrained("jky594176/BART1_GRU")
model = BartForCausalLM.from_pretrained("jky594176/recipe_BART1_GRU")
tokenizer = init_tokenizer("facebook/bart-base")
model = model.to('cuda')

In [19]:
torch.manual_seed(1)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
dir_bart1_gru =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)
  print("step:")
  for j in range(replicated_size):
    print(j, end = ", ")
    md = generate_recipe(raw_text)
    dir_bart1_gru[i].append(get_instr(md))


 Generating the recipe:  0
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  1
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  2
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  3
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  4
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  5
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  6
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  7
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  8
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  9
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  10
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  11
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  12
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  13
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  14
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  15
step:


In [20]:
print("Cos sim:", COS_SIM(df_eval_dir, dir_bart1_gru))
print("BLEU:", bleu_cal(df_eval_dir, dir_bart1_gru))
print("rouge-1:", rouge_cal(df_eval_dir, dir_bart1_gru,'rouge1'))
print("rouge-L:", rouge_cal(df_eval_dir, dir_bart1_gru,'rougeLsum'))

Cos sim: 0.5643583126180023
BLEU: 0.77208228998418
rouge-1: 0.3565509156364043
rouge-L: 0.2194203777934473


## BART1

In [21]:
model = BartForCausalLM.from_pretrained("jky594176/recipe_BART1")
model = model.to('cuda')

In [22]:
torch.manual_seed(1)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
dir_bart1 =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)
  print("step:")

  for j in range(replicated_size):
    print(j, end = ", ")
    md = generate_recipe(raw_text)
    dir_bart1[i].append(get_instr(md))


 Generating the recipe:  0
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  1
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  2
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  3
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  4
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  5
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  6
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  7
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  8
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  9
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  10
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  11
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  12
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  13
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  14
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  15
step:


In [23]:
print("Cos sim:", COS_SIM(df_eval_dir, dir_bart1))
print("BLEU:", bleu_cal(df_eval_dir, dir_bart1))
print("rouge-1:", rouge_cal(df_eval_dir, dir_bart1,'rouge1'))
print("rouge-L:", rouge_cal(df_eval_dir, dir_bart1,'rougeLsum'))

Cos sim: 0.5351948160757843
BLEU: 0.760743964253477
rouge-1: 0.3091233913976813
rouge-L: 0.18639506343143267


## GPT2

In [24]:
from transformers import AutoModelForCausalLM, GPT2Tokenizer

model = AutoModelForCausalLM.from_pretrained("jky594176/recipe_GPT2")
tokenizer = GPT2Tokenizer.from_pretrained("jky594176/recipe_GPT2")

In [25]:
model = model.to("cuda")

In [26]:
torch.manual_seed(1)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
dir_gpt2 =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)
  print("step:")

  for j in range(replicated_size):
    print(j, end = ", ")
    md = generate_recipe(raw_text)
    dir_gpt2[i].append(get_instr(md))


 Generating the recipe:  0
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  1
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  2
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  3
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  4
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  5
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  6
step:
0, 1, 2, Failed to generate, recipe's too long
3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  7
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  8
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  9
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  10
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  11
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  12
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  13
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  14
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 

In [27]:
print("cos similarity:", COS_SIM(df_eval_dir,dir_gpt2))
print("BLEU: ",bleu_cal(df_eval_dir, dir_gpt2))
print("rouge-1:", rouge_cal(df_eval_dir, dir_gpt2,'rouge1'))
print("rouge-L:", rouge_cal(df_eval_dir, dir_gpt2,'rougeLsum'))

cos similarity: 0.5389561959066712
BLEU:  0.770623806605888
rouge-1: 0.3006555958482678
rouge-L: 0.17599998217410337


## BART1+NN

In [28]:
model = BartForCausalLM.from_pretrained("jky594176/recipe_BART1")
tokenizer = init_tokenizer("facebook/bart-base")
model = model.to('cuda')

In [29]:
torch.manual_seed(0)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
dir_bart1_nn =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)
  print("step:")

  for j in range(replicated_size):
    print(j, end = ", ")
    md = generate_recipe(raw_text)
    dir_bart1_nn[i].append(get_instr(md))


 Generating the recipe:  0
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  1
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  2
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  3
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  4
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  5
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  6
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  7
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  8
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  9
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  10
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  11
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  12
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  13
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  14
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  15
step:


In [30]:
print("Cos sim:", COS_SIM(df_eval_dir, dir_bart1_nn))
print("BLEU:", bleu_cal(df_eval_dir, dir_bart1_nn))
print("rouge-1:", rouge_cal(df_eval_dir, dir_bart1_nn,'rouge1'))
print("rouge-L:", rouge_cal(df_eval_dir, dir_bart1_nn,'rougeL'))

Cos sim: 0.5342036210579831
BLEU: 0.7706537892876422
rouge-1: 0.31010366633561753
rouge-L: 0.1850874663184411


## BART2+GRU

The generation helper functions for BART2 is not the same with BART1. So re-define them here.

In [31]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, raw_text, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    ing_token_id = tokenizer.convert_tokens_to_ids(["<INPUT_END>"])[0]
    input_token = tokenizer.convert_tokens_to_ids(["<NEXT_INPUT>"])[0]

    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    # print(prepared_input)

    context_tokens = tokenizer.encode(prepared_input)[:-1]
    context_len = len(context_tokens)
    
    encoder_tokens = torch.tensor(context_tokens[3:], dtype=torch.long, device=device)
    encoder_tokens = encoder_tokens[encoder_tokens != input_token]

    context = torch.tensor(context_tokens, dtype=torch.long, device=device)
    # the encoder condition
    raw_text_tokens = encoder_tokens
    context = context.unsqueeze(0).repeat(num_samples, 1)
    raw_text_tokens = raw_text_tokens.unsqueeze(0).repeat(num_samples, 1)
    
    # genereated token
    start_token =  tokenizer.convert_tokens_to_ids(["<RECIPE_START>"])[0]
    # generated = torch.tensor(start_token, dtype=torch.long, device=device).reshape(1).unsqueeze(0)
    generated = context
    # print(generated.shape)
    
    with torch.no_grad():
        for _ in range(length):
            # inputs = {'input_ids': generated}
            outputs = model(input_ids = raw_text_tokens, decoder_input_ids = generated)
            # outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated, context_len

In [32]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):
  
    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    out, context_len = sample_sequence(
            model=model,
            raw_text=raw_text,
            tokenizer=tokenizer,
            length=512,
            device = "cuda"
        )
    out = out[0, context_len:].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      # print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      # print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

In [33]:
from transformers import BartForConditionalGeneration
# model = BartForCausalLM.from_pretrained("jky594176/BART1_GRU")
model = BartForConditionalGeneration.from_pretrained("jky594176/BART2_GRU")
tokenizer = init_tokenizer("facebook/bart-base")
model = model.to('cuda')

In [ ]:
torch.manual_seed(1)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
dir_bart2_gru =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)
  print("step:")

  for j in range(replicated_size):
    print(j, end = ", ")
    md = generate_recipe(raw_text)
    dir_bart2_gru[i].append(get_instr(md))


 Generating the recipe:  0
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  1
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  2
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  3
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  4
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  5
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  6
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  7
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  8
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  9
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  10
step:
0, 1, 2, 3, 4, Failed to generate, recipe's too long
5, 6, 7, 8, 9, 
 Generating the recipe:  11
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  12
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  13
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
 Generating the recipe:  14
step:
0, 1, 2, 3, 4, 5, 6, 7, 8, 

In [ ]:
print("Cos sim:", COS_SIM(df_eval_dir, dir_bart2_gru))
print("BLEU:", bleu_cal(df_eval_dir, dir_bart2_gru))
print("rouge-1:", rouge_cal(df_eval_dir, dir_bart2_gru,'rouge1'))
print("rouge-L:", rouge_cal(df_eval_dir, dir_bart2_gru,'rougeL'))

## BART2

In [ ]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, raw_text, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')

    context_tokens = tokenizer.encode(prepared_input)[0:-1]
    raw_text_tokens = tokenizer.encode(raw_text)[0:-1]

    context = torch.tensor(context_tokens, dtype=torch.long, device=device)
    raw_text_tokens = torch.tensor(raw_text_tokens, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    raw_text_tokens = raw_text_tokens.unsqueeze(0).repeat(num_samples, 1)
    # print(raw_text_tokens)
    # print(context)
    
    # genereated token
    start_token =  tokenizer.convert_tokens_to_ids(["<RECIPE_START>"])[0]
    # generated = torch.tensor(start_token, dtype=torch.long, device=device).reshape(1).unsqueeze(0)
    generated = context
    # print(generated.shape)
    
    with torch.no_grad():
        for _ in range(length):
            # inputs = {'input_ids': generated}
            outputs = model(input_ids = raw_text_tokens, decoder_input_ids = generated)
            # outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated

In [ ]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):
    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    context_tokens = tokenizer.encode(prepared_input)[0:-1]
    out = sample_sequence(
                model=model,
                raw_text=raw_text,
                tokenizer=tokenizer,
                length=512,
                device = 'cuda'
            )
    out = out[0, len(context_tokens):].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      # print(text)
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      # print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      # print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

In [ ]:
from transformers import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained("jky594176/recipe_bart2_v2")
model = model.to('cuda')

In [ ]:
torch.manual_seed(0)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
dir_bart2 =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)
  print("step:")

  for j in range(replicated_size):
    print(j, end = ", ")
    md = generate_recipe(raw_text)
    dir_bart2[i].append(get_instr(md))

In [ ]:
print("Cos sim:", COS_SIM(df_eval_dir, dir_bart2_gru))
print("BLEU:", bleu_cal(df_eval_dir, dir_bart2_gru))
print("rouge-1:", rouge_cal(df_eval_dir, dir_bart2_gru,'rouge1'))
print("rouge-L:", rouge_cal(df_eval_dir, dir_bart2_gru,'rougeL'))